## Day 9: Smoke Basin
[link](https://adventofcode.com/2021/day/9)

### Part 1

In [ ]:
#!value --name sampleRaw
2199943210
3987894921
9856789892
8767896789
9899965678

In [ ]:
#!value --name inputRaw --from-file ./data

In [ ]:

#!share sampleRaw --from value
#!share inputRaw --from value

In [ ]:
#load "../common.fsx"

let parse input = 
    read2d input
    |> Array2D.map (string >> int)
let sampleHeightMap = parse sampleRaw
let sampleHeightMapJagged = Array2D.toJagged sampleHeightMap
let actualHeightMap = parse inputRaw
let actualHeightMapJagged = Array2D.toJagged actualHeightMap

In [ ]:
let lowPoints (heightMap : int[,]) = 
    heightMap
    |> Array2D.mapi (fun i j value ->
                            let adj = 
                                [|
                                    // use slicing opertator in order not to check boundaries.
                                    // each line returns at most one adjacent element
                                    heightMap.[i-1..i-1, j]
                                    heightMap.[i+1..i+1, j]
                                    heightMap.[i, j-1..j-1]
                                    heightMap.[i, j+1..j+1]
                                |] |> Array.collect id
                            
                            if (value < Array.min adj)
                            then Some (i,j,value)
                            else None
                        )
    |> Seq.cast<(int*int*int) option>
    |> Seq.choose id
    |> Array.ofSeq

let sampleLowPoints = lowPoints sampleHeightMap
let actualLowPoints = lowPoints actualHeightMap

In [ ]:
<script src="./common.mjs"/>

In [ ]:
<canvas id="sampleHeightMapJaggedWithLow"></canvas>

In [ ]:
#!share --from fsharp sampleHeightMapJagged
#!share --from fsharp sampleLowPoints

var canvas = document.getElementById("sampleHeightMapJaggedWithLow");

var size = prepareCanvas(sampleHeightMapJagged, canvas, 200)
drawMap(sampleHeightMapJagged, canvas, size);
drawPoints(sampleLowPoints, "black", canvas, size);


In [ ]:
<canvas id="actualHeightMapJaggedWithLow"></canvas>

In [ ]:
#!share --from fsharp actualLowPoints
#!share --from fsharp actualHeightMapJagged
var canvas = document.getElementById("actualHeightMapJaggedWithLow");
var size = prepareCanvas(actualHeightMapJagged, canvas, 600)
drawMap(actualHeightMapJagged, canvas, size);
drawPoints(actualLowPoints, "black", canvas, size);

In [ ]:
let riskLevel lowPoints = 
    lowPoints
    |> Array.map (fun (_,_,value) -> value)
    |> Array.map ((+) 1)
    |> Array.sum
    
sampleLowPoints |> riskLevel |> display
actualLowPoints |> riskLevel |> display

15

417

### Part 2

In [ ]:
let (|Empty|NotEmpty|) set =
    if (Set.isEmpty set) then Empty
    else
        let head = Seq.head set
        NotEmpty (head, (Set.remove head set))
let basins (heightMap : int[,]) = 
    let allIndices = 
        heightMap |> Array2D.mapi (fun i j value -> match value with | 9 -> None | _ -> Some (i,j))
        |> Seq.cast<(int*int) option>
        |> Seq.choose id
        |> Set.ofSeq
        
    let rec findBasin i j indicesLeft basinAcc =
        if (not <| Set.contains (i,j) indicesLeft) then indicesLeft, basinAcc
        else 
            let newIndices = indicesLeft |> Set.remove (i,j)
            (newIndices, (i,j)::basinAcc)
            ||> findBasin (i-1) j
            ||> findBasin i (j-1)
            ||> findBasin (i+1) j
            ||> findBasin i (j+1)

    let rec find (indicesLeft:Set<int*int>) (basinsAcc : (int*int) list list) =
        if (Set.isEmpty indicesLeft) then basinsAcc
        else
            let (i,j) = Seq.head indicesLeft
            let (newIndicesLeft, basin) = findBasin i j indicesLeft []
            basin :: (find newIndicesLeft [])
     
    find allIndices [] |> List.filter (not << List.isEmpty)

In [ ]:
let sampleBanisPoints = basins sampleHeightMap;
let actualBanisPoints = basins actualHeightMap;

In [ ]:
<canvas id="sampleBanisPoints"></canvas>

In [ ]:
#!share --from fsharp sampleBanisPoints

var canvas = document.getElementById("sampleBanisPoints");
var size = prepareCanvas(sampleHeightMapJagged, canvas, 200)
sampleBanisPoints.forEach(basin => {
    drawPoints(basin, 'hsla(' + (Math.random() * 360) + ', 80%, 50%, 1)', canvas, size);
})

In [ ]:
<canvas id="actualBanisPoints"></canvas>

In [ ]:
#!share --from fsharp actualBanisPoints

var canvas = document.getElementById("actualBanisPoints");
var size = prepareCanvas(actualHeightMapJagged, canvas, 500)
actualBanisPoints.forEach(basin => {
    drawPoints(basin, 'hsla(' + (Math.random() * 360) + ', 80%, 50%, 1)', canvas, size);
})

In [ ]:
let private multiplyTop3 basins = 
    basins
    |> Seq.sortByDescending List.length
    |> Seq.take 3
    |> Seq.map List.length
    |> Seq.reduce (*)

multiplyTop3 (basins sampleHeightMap) |> display
multiplyTop3 (basins actualHeightMap) |> display

1134

1148965